In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Load the image
image_path = 'apple.jpeg'  # replace with your image path
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Step 1: Apply Otsu's thresholding to convert the image to binary
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Step 2: Remove noise using morphological operations (opening)
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)

# Step 3: Dilate the binary image to get sure background
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Step 4: Distance transform to find sure foreground regions
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Step 5: Find unknown region by subtracting sure foreground from sure background
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Step 6: Label markers
_, markers = cv2.connectedComponents(sure_fg)

# Step 7: Add 1 to all markers to distinguish between background and other regions
markers = markers + 1

# Mark the region of unknown (background) with 0
markers[unknown == 255] = 0

# Step 8: Apply watershed algorithm
markers = cv2.watershed(img, markers)

# Step 9: Boundaries are marked with -1 in the marker image, so highlight the boundaries
img[markers == -1] = [255, 0, 0]  # marking watershed boundaries with red color

# Display results
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.title('Segmented Image')

plt.subplot(122)
plt.imshow(markers, cmap='gray')
plt.title('Markers')

plt.tight_layout()
plt.show()
